In [71]:
library(caret)
country.code <- 'us'  # use yours
url.pattern <- 'https://'  # use http if you want
repo.data.frame <- subset(getCRANmirrors(), CountryCode == country.code & grepl(url.pattern, URL))
options(repos = repo.data.frame$URL)

In [159]:
drop_cols <- function(df, cols_to_drop){
    for (col in cols_to_drop){
        df[[col]] = NULL
    }
    return(df)
}

source <- read.csv('cpi.csv')
    
country_map <- read.csv('country_map.csv')
cols_to_drop = c('Methodology', 'Total.Quota', 'Max.Feasibility', 'Internet.Penetration', 'Recommended...Online', 'Translation.with.Edit', 'Comment.One', 'Comment.Two', 'Comment.Three', 'Country', 'OrigCountryCount')

df <- drop_cols(merge(source, country_map, on='Country'), cols_to_drop)

df$Language <- ifelse(df$Language != 'English', 'Other', 'English')

df$Incidence <- df$Incidence * 100
df$I2 <- (df$Incidence ^ 2)


head(df)

set.seed(998)
inTraining <- createDataPartition(df$CPI, p = .75, list = FALSE)
train_df <- df[inTraining, ]
test_df <- df[-inTraining, ]

Survey.Length,Incidence,CPI,Language,Region,I2
15,60,6.0,Other,South America,3600
30,60,9.0,Other,South America,3600
10,20,15.0,English,Other,400
10,35,13.0,English,Other,1225
15,10,22.0,English,Other,100
15,20,15.5,English,Other,400


In [160]:
set.seed(42)

model <- train(
    CPI ~ ., train_df,
    method = 'lm',
    trControl = trainControl(
        method = 'cv', number = 20,
        repeats = 5, verboseIter = TRUE
    )
)

print (model)

+ Fold01: intercept=TRUE 
- Fold01: intercept=TRUE 
+ Fold02: intercept=TRUE 
- Fold02: intercept=TRUE 
+ Fold03: intercept=TRUE 
- Fold03: intercept=TRUE 
+ Fold04: intercept=TRUE 
- Fold04: intercept=TRUE 
+ Fold05: intercept=TRUE 
- Fold05: intercept=TRUE 
+ Fold06: intercept=TRUE 
- Fold06: intercept=TRUE 
+ Fold07: intercept=TRUE 
- Fold07: intercept=TRUE 
+ Fold08: intercept=TRUE 
- Fold08: intercept=TRUE 
+ Fold09: intercept=TRUE 
- Fold09: intercept=TRUE 
+ Fold10: intercept=TRUE 
- Fold10: intercept=TRUE 
+ Fold11: intercept=TRUE 
- Fold11: intercept=TRUE 
+ Fold12: intercept=TRUE 
- Fold12: intercept=TRUE 
+ Fold13: intercept=TRUE 
- Fold13: intercept=TRUE 
+ Fold14: intercept=TRUE 
- Fold14: intercept=TRUE 
+ Fold15: intercept=TRUE 
- Fold15: intercept=TRUE 
+ Fold16: intercept=TRUE 
- Fold16: intercept=TRUE 
+ Fold17: intercept=TRUE 
- Fold17: intercept=TRUE 
+ Fold18: intercept=TRUE 
- Fold18: intercept=TRUE 
+ Fold19: intercept=TRUE 
- Fold19: intercept=TRUE 
+ Fold20: in

In [161]:
p <- predict(model, test_df)
actuals <- test_df$CPI
error <- (p - actuals)

sqrt(mean(error ^ 2))

[1] 2.647392

In [162]:
summary(model)


Call:
lm(formula = .outcome ~ ., data = dat)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.3360 -1.3563 -0.4513  0.7462 21.9401 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           12.683322   0.494146  25.667  < 2e-16 ***
Survey.Length          0.258961   0.012731  20.342  < 2e-16 ***
Incidence             -0.350657   0.015976 -21.949  < 2e-16 ***
LanguageOther          0.475050   0.272463   1.744 0.081575 .  
RegionEurope          -0.774437   0.214421  -3.612 0.000321 ***
`RegionNorth America` -0.304860   0.309435  -0.985 0.324779    
RegionOther            4.108647   0.448757   9.156  < 2e-16 ***
`RegionSouth America` -0.185120   0.275396  -0.672 0.501629    
I2                     0.002305   0.000181  12.737  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.501 on 916 degrees of freedom
Multiple R-squared:  0.7364,	Adjusted R-squared:  0.7341 
F-statistic: 319.8 on 